# Roboflow에서 labeling한 COCO json 정보 시각화 및 mask 파일 변환

In [40]:
import cv2
from pycocotools.coco import COCO
import numpy as np

# COCO API 초기화
coco = COCO('rf_coco_seg.json')

# 첫 번째 이미지 정보 로드
img_info = coco.loadImgs(0)[0]

# 어노테이션 ID 및 정보 가져오기
ann_ids = coco.getAnnIds(imgIds=img_info['id'])
anns = coco.loadAnns(ann_ids)

# 클래스 수에 따라 색상 맵 생성 (시각화를 위한)
color_map = np.random.randint(0, 255, (len(coco.getCatIds()), 3))

# 그레이스케일 마스크 생성 (클래스 ID를 저장)
mask_gray = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

# RGB 마스크 생성 (시각화를 위한)
mask_rgb = np.zeros((img_info['height'], img_info['width'], 3), dtype=np.uint8)

for ann in anns:
    category_id = ann['category_id']
    
    # 그레이스케일 마스크에 클래스 ID 저장
    category_mask = coco.annToMask(ann)
    mask_gray[category_mask > 0] = category_id  # 클래스 ID로 설정
    
    # RGB 마스크에 색상 적용
    category_color = color_map[category_id % len(color_map)]
    mask_rgb[category_mask > 0] = category_color

# OpenCV를 사용하여 그레이스케일 마스크 저장
cv2.imwrite("mask_temp_gray.png", mask_gray)

# OpenCV를 사용하여 RGB 마스크 저장 (시각화용)
cv2.imwrite("mask_temp_rgb.png", mask_rgb)

# # 선택적으로 원본 이미지 위에 RGB 마스크 오버레이하기
# image = cv2.imread(img_info['file_name'])
# overlay = cv2.addWeighted(image, 0.7, mask_rgb, 0.3, 0)
# cv2.imwrite("overlay_temp.png", overlay)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


True

# grayscale의 저장된 masking 이미지 확인

In [26]:
import cv2

# 저장된 masking 이미지의 채널 확인(RGB 채널이 아닌 Gray scale이 맞는지 확인)
gray_img = cv2.imread('mask_temp_gray.png', cv2.IMREAD_GRAYSCALE)
gray_img.shape

(3024, 4032)

> 저장된 masking 이미지의 채널 확인(RGB 채널이 아닌 Gray scale이 맞는지 확인)

In [ ]:
import cv2

# 이미지 불러오기
gray_img = cv2.imread('mask_temp_gray.png', cv2.IMREAD_GRAYSCALE)
gray_scaled = gray_img * int(255.0/gray_img.max())    # class별 mask scale을 255로 맞춤으로써 시각적으로 보이도록 만듦
# >>> 현재 시각화 편의를 위해 gray_img의 max() 값을 기준으로 나누지만, class 갯수가 정해져 있을 경우 이를 고려하여 값이 게산되어야 함함

# 원본 이미지 크기 출력
print(f"Original Image Shape: {gray_img.shape}")  # (height, width, channels)

# 전체 비율 설정
scale_factor = 0.25  # 전체 비율의 25%

# 비율에 따라 이미지 크기 조절
resized_gray = cv2.resize(gray_img, dsize=(0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
resized_scaled_gray = cv2.resize(gray_scaled, dsize=(0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

# 조정된 이미지 크기 출력
print(f"Resized Image Shape: {resized_gray.shape}")  # (new_height, new_width, channels)

# 조정된 이미지 표시
cv2.imshow("Resized Origin Gray Image", resized_gray)
cv2.imshow("Resized Scaled Gray Image", resized_scaled_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()


Original Image Shape: (3024, 4032)
Resized Image Shape: (756, 1008)
